In [40]:
# predict the next rssi value from current time value
import tensorflow as tf
import numpy as np
import random
import pandas as pd
%matplotlib inline

In [58]:
def make_mini_batch(train_data, size_of_mini_batch, length_of_sequences):
    inputs  = np.empty(0)
    outputs = np.empty(0)
    for _ in range(size_of_mini_batch):
        index   = random.randint(0, len(train_data) - length_of_sequences)
        part    = train_data[index:index + length_of_sequences]
        inputs  = np.append(inputs, part[:, 0])
        outputs = np.append(outputs, part[-1, 2])
    inputs  = inputs.reshape(-1, length_of_sequences, 1)
    outputs = outputs.reshape(-1, 1)
    return (inputs, outputs)

In [59]:
def make_prediction_initial(train_data, index, length_of_sequences):
    return train_data[index:index + length_of_sequences, 0]

In [82]:
#''' pred area judgement from rssi
train_data_path             = "./data.npy"
num_of_input_nodes          = 1
num_of_hidden_nodes         = 20
num_of_output_nodes         = 1
length_of_sequences         = 40
num_of_training_epochs      = 20

length_of_initial_sequences = 40
num_of_prediction_epochs    = 0

size_of_mini_batch          = 10
learning_rate               = 0.001
forget_bias                 = 1.0

In [79]:
train_data = np.load(train_data_path)
#print("train_data:", train_data[:10])
# normalize
train_data[:,0] = (train_data[:,0] + 60.0)/30
train_data[:10]

array([[-0.7, 30.0, False],
       [-0.7, 29.83333333, False],
       [-0.6666666666666666, 29.66666667, False],
       [-0.6, 29.5, False],
       [-0.5, 29.33333333, False],
       [-0.43333333333333335, 29.16666667, False],
       [-0.36666666666666664, 29.0, False],
       [-0.3333333333333333, 28.83333333, False],
       [-0.3, 28.66666667, False],
       [-0.23333333333333334, 28.5, False]], dtype=object)

In [80]:
# random seed fix
random.seed(0)
np.random.seed(0)
tf.set_random_seed(0)

In [81]:
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

with tf.Graph().as_default():
    input_ph      = tf.placeholder(tf.float32, [None, length_of_sequences, num_of_input_nodes], name="input")
    supervisor_ph = tf.placeholder(tf.float32, [None, num_of_output_nodes], name="supervisor")
    istate_ph     = tf.placeholder(tf.float32, [None, num_of_hidden_nodes * 2], name="istate")

    with tf.name_scope("inference") as scope:
        weight1_var = tf.Variable(tf.truncated_normal([num_of_input_nodes, num_of_hidden_nodes], stddev=0.1), name="weight1")
        weight2_var = tf.Variable(tf.truncated_normal([num_of_hidden_nodes, num_of_output_nodes], stddev=0.1), name="weight2")
        bias1_var   = tf.Variable(tf.truncated_normal([num_of_hidden_nodes], stddev=0.1), name="bias1")
        bias2_var   = tf.Variable(tf.truncated_normal([num_of_output_nodes], stddev=0.1), name="bias2")

        weight1_hist = tf.histogram_summary("layer1/weights", weight1_var)
        weight2_hist = tf.histogram_summary("layer2/weights", weight2_var)
        bias1_hist = tf.histogram_summary("layer1/biases", bias1_var)
        bias2_hist = tf.histogram_summary("layer2/biases", bias2_var)

        in1 = tf.transpose(input_ph, [1, 0, 2])         # (batch, sequence, data) -> (sequence, batch, data)
        in2 = tf.reshape(in1, [-1, num_of_input_nodes]) # (sequence, batch, data) -> (sequence * batch, data)
        in3 = tf.matmul(in2, weight1_var) + bias1_var
        in4 = tf.split(0, length_of_sequences, in3)     # sequence * (batch, data)

        cell = tf.nn.rnn_cell.BasicLSTMCell(num_of_hidden_nodes, forget_bias=forget_bias)
        rnn_output, states_op = tf.nn.rnn(cell, in4, initial_state=istate_ph)
        out1 = tf.matmul(rnn_output[-1], weight2_var) + bias2_var
        output_op = tf.nn.sigmoid(out1)

    with tf.name_scope("loss") as scope:
        #square_error = tf.reduce_mean(tf.square(output_op - supervisor_ph))
        #loss_op      = square_error
        log_likelihood = tf.reduce_mean(tf.reduce_sum(supervisor_ph * tf.log(output_op) + (1 - supervisor_ph) * tf.log(1 - output_op), reduction_indices=[1]))
        loss_op = -log_likelihood
        tf.scalar_summary("loss", loss_op)

    with tf.name_scope("training") as scope:
        training_op = optimizer.minimize(loss_op)

    summary_op = tf.merge_all_summaries()
    init = tf.initialize_all_variables()

    with tf.Session() as sess:
        saver = tf.train.Saver()
        summary_writer = tf.train.SummaryWriter("data", graph=sess.graph)
        sess.run(init)

        for epoch in range(num_of_training_epochs):
            inputs, supervisors = make_mini_batch(train_data, size_of_mini_batch, length_of_sequences)

            train_dict = {
                input_ph:      inputs,
                supervisor_ph: supervisors,
                istate_ph:     np.zeros((size_of_mini_batch, num_of_hidden_nodes * 2)),
            }
            sess.run(training_op, feed_dict=train_dict)

            if (epoch + 1) % 10 == 0:
                summary_str, train_loss = sess.run([summary_op, loss_op], feed_dict=train_dict)
                summary_writer.add_summary(summary_str, epoch)
                print("train#%d, train loss: %e" % (epoch + 1, train_loss))
                
                # eval accuracy
                inputs, supervisors = make_mini_batch(train_data, size_of_mini_batch, length_of_sequences)
                pred_dict = {
                    input_ph:      inputs,
#                    supervisor_ph: supervisors,
                    istate_ph:     np.zeros((size_of_mini_batch, num_of_hidden_nodes * 2)),
                }
                print(sess.run(output_op, feed_dict=pred_dict))
                

        outputs = np.empty(0)
        states  = np.zeros((num_of_hidden_nodes * 2)),

        for epoch in range(num_of_prediction_epochs):
            inputs  = make_prediction_initial(train_data, epoch, length_of_initial_sequences)
            pred_dict = {
                input_ph:  inputs.reshape((1, length_of_sequences, 1)),
                istate_ph: states,
            }
            output, states = sess.run([output_op, states_op], feed_dict=pred_dict)
            print("prediction#%d, output: %f" % (epoch + 1, output))

            #inputs  = np.delete(inputs, 0)
            #inputs  = np.append(inputs, output)
            outputs = np.append(outputs, output)

#        print("outputs:", outputs)
#        np.save("output.npy", outputs)
#        saver.save(sess, "data/model")

train#10, train loss: 7.072002e-01
[[ 0.5118348 ]
 [ 0.51195288]
 [ 0.51189911]
 [ 0.51176691]
 [ 0.51209664]
 [ 0.51193672]
 [ 0.51207906]
 [ 0.5118348 ]
 [ 0.51211983]
 [ 0.51211983]
 [ 0.51160997]
 [ 0.51212358]
 [ 0.51207709]
 [ 0.51222807]
 [ 0.51195753]
 [ 0.51199961]
 [ 0.5118475 ]
 [ 0.51167184]
 [ 0.51167339]
 [ 0.51207906]
 [ 0.51222885]
 [ 0.51157731]
 [ 0.5118348 ]
 [ 0.51202023]
 [ 0.51186919]
 [ 0.51222885]
 [ 0.51209664]
 [ 0.51189715]
 [ 0.51191193]
 [ 0.51160568]
 [ 0.51176691]
 [ 0.51212358]
 [ 0.51160997]
 [ 0.51209682]
 [ 0.51178408]
 [ 0.51193881]
 [ 0.5118348 ]
 [ 0.51178408]
 [ 0.51197547]
 [ 0.51203376]
 [ 0.51199591]
 [ 0.5117994 ]
 [ 0.51196635]
 [ 0.5121085 ]
 [ 0.51193637]
 [ 0.51148903]
 [ 0.51190263]
 [ 0.51190263]
 [ 0.51220602]
 [ 0.51157331]
 [ 0.51174092]
 [ 0.51196325]
 [ 0.51166427]
 [ 0.5118348 ]
 [ 0.51209682]
 [ 0.51213783]
 [ 0.5121184 ]
 [ 0.51167554]
 [ 0.5121184 ]
 [ 0.5122304 ]
 [ 0.51199591]
 [ 0.5115813 ]
 [ 0.51175463]
 [ 0.51211959]
 [ 0.